In [6]:
from pathlib import Path
import pandas as pd
import boto3
from io import StringIO

In [8]:
client = boto3.client('s3')
bucket_name = 'columbia-project-resources'
object_key = 'bitcoin.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

bitcoin_df = pd.read_csv(StringIO(csv_string))
bitcoin_df.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,9/17/14,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,9/18/14,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,9/19/14,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,9/20/14,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,9/21/14,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0
